In [3]:
import json
import urllib.request
from datetime import datetime
from datetime import timedelta
import re

import pandas as pd

#### twitch v5 api

In [7]:
#클립 수집
#스트리머가 전체 영상에서 일부 장면을 편집하여 업로드한 내용
def collectClip(channel, lim, Clientid, File):
    url = "https://api.twitch.tv/kraken/clips/top?channel="+channel+"&limit="+str(lim)
    req = urllib.request.Request(url, headers = {"Client-ID": Clientid, "Accept" : "application/vnd.twitchtv.v5+jason"})
    u = urllib.request.urlopen(req)
    c = u.read().decode('utf-8')
    js = json.loads(c)
    
    collectChat(js, lim, Clientid, File)

def collectChat(j, limit, clientid, f):
    for num in range(int(limit)):
        id = j['clips'][num]['vod']['id']
        offset = j['clips'][num]['vod']['offset']
        duration = j['clips'][num]['duration']
        
        cursor = ""
        count = 0
        
        while(1):
            try:
                url2 = ""
                if count == 0:
                    url2 = "https://api.twitch.tv/kraken/videos/"+str(id)+"/comments?content_offset_seconds="+str(offset)
                else:
                    url2 = "https://api.twitch.tv/kraken/videos/"+str(id)+"/comments?cursor="+str(cursor)
                req2 = urllib.request.Request(url2, headers = {"Client-ID": clientid, "Accept" : "application/vnd.twitchtv.v5+json"})
                u2 = urllib.request.urlopen(req2)
                c2 = u2.read().decode('utf-8')
                j2 = json.loads(c2)
                endCount = 0
    
                try:
                    for number, com in enumerate(j2['comments']):
                        dateString = j2['comments'][number]['created_at']
                        if "." in dateString:
                            dateString = re.sub(r".[0-9]+Z","Z", dateString)
                        date = datetime.strptime(dateString, "%Y-%m-%dT%H:%M:%SZ")
                        if (int(offset) + int(duration)) < int(j2['comments'][number]['content_offset_seconds']):
                            endCount = 1
                            break
                        else:
                            f.write(str(j['clips'][num]['title']) + "\t" +
                                    str(j['clips'][num]['game']) + "\t" +
                                    str(j['clips'][num]['views']) + "\t" +
                                    str(j['clips'][num]['duration']) + "\t" +
                                    str(j['clips'][num]['vod']['url']) + "\t" +
                                    str(date + timedelta(hours=9)) + "\t" +
                                    str(j2['comments'][number]['commenter']['name']) + "\t" +
                                    str(j2['comments'][number]['message']['body']) + "\n")

                except Exception as e:
                    print(e)

                if endCount == 1:
                    break

                if j2['_next']:
                    cursor = j2['_next']

                count = count + 1

            except Exception as e:
                print(e)

In [9]:
if __name__ == "__main__":
    file = open("clipchat.txt", "w", encoding="utf-8")
    Channel = "hanryang1125"
    Limit = 5
    ClientId = "no5o3zszanjsu3q6coq8tvi2vy38jh" # Client id 추가
    collectClip(Channel, Limit, ClientId, file)
    file.close()

In [18]:
a=pd.read_csv('clipchat.txt', sep='\t', names=['Clip_name','game_category','view','duration','vod_url','chat_time','comment_user','comments'])
a

,Clip_name,category,view,duration,vod_url,time,comment_user,comments
0,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:34,ansdbghks,공중파보다 잘나가지 ㅋㅋ
1,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:34,wogus8989,인방으로 일본애니보면요
2,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:34,ssambahong__,ㅁㅏ찌
3,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:34,lurenze,대중하
4,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:35,eztwoon,도넘은 인터넷방송
5,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:35,marmizar,시대 흐름 못따라오는 늙은이들이야
6,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:36,holic07,인싸가 다 뺏어가 ㅠㅠ
7,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:36,jdw8378,유튜브가 인터넷방송이지 ㅋㅋ
8,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:37,zgreenbee7,마리텔 종영
9,명언 ) 인터넷방송을 무시하는건 시대적으로 뒤떨어진 꼰대나 마찬가지다,Just Chatting,24276,59.98,https://www.twitch.tv/videos/539958449?t=44m54s,2020-01-22 20:46:37,mestation0525,유튜브도 곧 인방이야


In [21]:
#a.to_csv('testChat.csv', index=False, encoding="utf-8")
#a.to_excel('testChat.xlsx', index=False)